In [1]:
import os
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import time

In [2]:
raw_data = pd.read_csv('../data/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})

In [34]:
batch_size = 150
seq_len = 50
dropout = 0.2
learning_rate = 0.001

is_cuda = torch.cuda.is_available()
# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("training on GPU")
else:
    device = torch.device("cpu")
    print("training on CPU")

training on CPU


In [28]:
class Data(Dataset):
    def __init__(self, df, window_size=1000, sequence_len=seq_len):
        self.rows = df.shape[0] // (window_size*sequence_len)
        self.data, self.labels = [], []
        print(self.rows)
        print(df.shape[0])
        
        for s in range(self.rows):
            seg = df.iloc[s*window_size*sequence_len: (s+1)*window_size*sequence_len]
            x = seg.acoustic_data.values
            y = seg.time_to_failure.values[-1]
            self.data.append(create_X(x))
            self.labels.append(y)
            
    def __len__(self):
        return self.rows
    
    def __getitem__(self, idx):
        return (
            torch.from_numpy(self.data[idx].astype(np.float32)),
            self.labels[idx]
        )


def feature_extraction(time_step):
    return np.c_[time_step.mean(axis=1), 
                 np.percentile(np.abs(time_step), q=[0, 25, 50, 75, 100], axis=1).T,
                 time_step.std(axis=1)]



def create_X(x, window_size=1000, seq_len=seq_len):
    X = x.reshape(seq_len, -1)
    return np.c_[feature_extraction(X),
                 feature_extraction(X[:, -window_size // 10:]),]

In [5]:
class GRU(nn.Module):
    
    def __init__(self, D_in, H, D_out, n_layers, dropout = 0.2):
        super(GRU, self).__init__()
        self.hidden_size = H
        self.n_layers = n_layers
        print("hidden_size" + str(H))
        self.gru = nn.GRU(input_size = D_in, hidden_size = H,
                          num_layers = n_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(H, D_out)
        self.relu = nn.ReLU()
    
    def forward(self, x, h):
        out, h = self.gru(x,h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h

    def init_hidden(self, batch_size):
        print(batch_size)
        is_cuda = torch.cuda.is_available()
        if is_cuda:
            device = torch.device("cuda")
        else:
            device = torch.device("cpu")
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_size).zero_().to(device)
        print(hidden.shape)
        return hidden

In [35]:
def train_model(train_loader, lr = 0.01, hidden_dim = 256, epochs = 10):
    input_size = next(iter(train_loader))[0].shape[2]
    output_size = 1
    n_layers = 2
    model = GRU(input_size, hidden_dim, output_size, n_layers)
    
    h = model.init_hidden(batch_size)
    model.to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    epoch_times = []

    print('=========> Starting training')
    history = []
    for epoch in range(epochs):
        for i, (data, labels) in enumerate(train_loader):
            if data.shape[0] != batch_size:
                continue
            outputs, h = model(data, h)
            loss = criterion(outputs, labels.view(-1,1))
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()
            if i % 10 == 0:
                print(f'[Epoch {epoch+1}/2, Step {i}/{train_loader.__len__()}]  loss: {loss.item(): .4f}')
        history.append((epoch+1,loss.item()))
    return model, history

In [7]:
def evaluate(model, data_loader):
    h = model.init_hidden(batch_size)
    out = []
    targets = []
    model.eval()
    diff = []
    with torch.no_grad():
        for x, y in data_loader:
            h = h.data
            out, h = model(x.to(device).float(), h)
            out = out.reshape(-1)
            print(out.shape)
            diff.append(y - out)
            
    print(diff[0])
    MAE = abs(sum(diff))/len(diff)
    
    print("MAE: {}%".format(MAE*100))
    return MAE

In [9]:
def loss_curve(history):
    x_val = [x[0] for x in history]
    y_val = [x[1] for x in history]

    plt.title('model MAE')
    plt.ylabel('LOSS')
    plt.xlabel('epoch')

    plt.plot(x_val,y_val)
    plt.plot(x_val,y_val,'or')
    plt.show()
    plt.savefig('GRU_loss.png')

In [10]:
mask = np.random.rand(len(raw_data)) < 0.8
train = raw_data[mask]
test = raw_data[~mask]

In [25]:
train_set = Data(train)
test_set = Data(test)
train_loader = DataLoader(train_set,shuffle=False, batch_size=batch_size)
test_loader = DataLoader(test_set, shuffle=False, batch_size=batch_size) 

10066
503311929
2516
125833551


In [36]:
model,history = train_model(train_loader)

hidden_size256
150
torch.Size([2, 150, 256])
=========> Starting training


In [ ]:
#save the model
save_path = None # change this to your desired path
torch.save(model, save_path)

In [ ]:
gru_MAE = evaluate(model, test_loader)

In [ ]:
import matplotlib. pyplot as plt 
loss_curve(history)

## generate submission

In [58]:
from tqdm import tqdm

output = []
submission = pd.read_csv('../data/sample_submission.csv', index_col='seg_id', dtype={"time_to_failure": np.float32})
model.eval()
h = model.init_hidden(1)
for i, seg_id in enumerate(tqdm(submission.index)):
  #  print(i)
    seg = pd.read_csv('../data/test/' + seg_id + '.csv')
    raw_x = seg['acoustic_data'].values
    X = create_X(raw_x)
    X = torch.from_numpy(X).float()
    out,h = model(X.view(1,seq_len,14),h)  # 14 is the number of features
    print(out.data[0][0].item())
    output.append(out.data[0][0].item())
submission['time_to_failure'] = output
submission.to_csv('submission-GRU.csv')


  0%|          | 0/2624 [00:00<?, ?it/s]


1
torch.Size([2, 1, 256])
lol
tensor([[4.1193]], grad_fn=<AddmmBackward>)
